In [2]:
using PyCall
using PyPlot
using FFTW
using LinearAlgebra
using JLD2
using YAML
# FFTW.set_num_threads(16)

@pyimport mpl_toolkits.axes_grid1 as axgrid
plt.style.use("dark_background")
# plt.style.use("default")
plt.rc("font", family="STIXGeneral")
plt.rc("font", size=22)
plt.rc("mathtext", fontset="stix")

mpl = pyimport("matplotlib")
np = pyimport("numpy")

config = YAML.load_file("../config.yaml")

Dict{Any, Any} with 16 entries:
  "DM_particle_mass_Dark" => 0.00472716
  "run_name"              => "density_cube_snap_099_full_dm"
  "xmin"                  => 0.0
  "num_scales"            => 10
  "GAS_particle_mass_TNG" => 0.000743736
  "output_directory"      => "/global/cscratch1/sd/james12/NEXUS_analysis/"
  "nx"                    => 1024
  "N_GAS"                 => 15625000000
  "num_snaps"             => 1
  "snapfile_root"         => 99
  "algorithm"             => "NEXUS+"
  "input_directory"       => "/global/cscratch1/sd/james12/Illustris_TNG_Data/T…
  "simulation_type"       => "Dark"
  "DM_particle_mass_TNG"  => 0.00398343
  "N_DM"                  => 15625000000
  "xmax"                  => 205000.0

In [3]:
#load in data and reduce size for example
@load config["input_directory"] * config["run_name"] * ".jld2" den

# np.save("../../../../output_files/test_99.npy", den)

1-element Vector{Symbol}:
 :den

In [ ]:
den = den[1:256, 1:256, 1:256]

256×256×256 Array{Float32, 3}:
[:, :, 1] =
 0.774086   0.460303   0.350756   …   0.0772807   0.0734285   0.0868222
 0.462835   0.647863   0.663849       0.0936511   0.0905545   0.0554346
 1.2541     8.50616    6.9514         0.0748076   0.0752607   0.0594153
 1.03841    2.12358    1.83277        0.103594    0.0797793   0.0709353
 0.650128   0.485225   0.59411        0.0979      0.128301    0.081677
 0.54259    0.502665   0.444703   …   0.0583678   0.0757361   0.079466
 0.203929   0.129897   0.109668       0.0439333   0.075645    0.0544217
 0.0524556  0.0501958  0.0524891      0.0518235   0.0373275   0.0264314
 0.0585399  0.0138464  0.0297382      0.108884    0.0475167   0.0210947
 0.0290289  0.0592201  0.104856       0.091613    0.0814836   0.0431785
 0.0946684  0.0997671  0.0713173  …   0.118178    0.0843563   0.0721431
 0.179084   0.211307   0.114244       0.0704223   0.0850478   0.0677275
 0.421284   0.511061   0.369406       0.0774507   0.0374965   0.0557047
 ⋮                     

In [21]:
den

512×512×512 Array{Float32, 3}:
[:, :, 1] =
 0.164515   0.0        0.972886   …  0.0        0.0         0.0
 0.0241716  0.219159   1.79951       0.0        0.0         0.0
 0.153413   2.38871    3.68746       0.0        0.00982262  1.31862
 0.124503   2.14765    0.586298      0.0        0.00328088  0.440438
 0.341669   2.24036    0.526099      0.0        0.0         0.0
 0.301784   1.36171    0.0        …  0.0        0.0         0.0
 0.0        0.0        1.96672       0.0        0.0         0.0
 0.0        0.0        1.02979       0.0        0.0         0.0
 0.0        0.0        0.0           0.460942   0.218619    0.0
 0.0        0.0        0.0           0.0542262  0.0257188   0.0
 0.0        0.424646   1.38357    …  0.0        0.0         0.0
 0.0        0.354632   1.15546       0.0        0.0625293   0.294462
 0.0775411  0.0        0.0           0.0        0.107306    0.505325
 ⋮                                ⋱             ⋮           
 0.0        0.0        0.0        …  0.0     

In [4]:
#use  slide 57 volume fractions for the quantiles (0.001, 0.06, 0.18, 0.77) for Q1, Q2, Q3, Q4
q1, q2, q3 = np.quantile(den, [0.05, 0.8, 0.95])

3-element Vector{Float64}:
 0.019333020318299532
 0.3102044463157654
 1.447799175977707

In [ ]:
structure_class = np.zeros(shape=size(den))
structure_class[np.where (den<q1)] = 0
structure_class[np.where ((den<q2) & (den>q1))] = 1
structure_class[np.where ((den<q3) & (den>q2))] = 2
structure_class[np.where (den>q3)] = 3


LoadError: MethodError: no method matching +(::Vector{Int64}, ::Int64)
For element-wise addition, use broadcasting with dot syntax: array .+ scalar
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  +([91m::T[39m, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:87
[0m  +([91m::Base.TwicePrecision[39m, ::Number) at twiceprecision.jl:267
[0m  ...

In [12]:
q1_bool = structure_class == 0
q2_bool = structure_class == 1
q3_bool = structure_class == 2
q4_bool = structure_class == 3

q1_bool = q1_bool.astype(int)
q2_bool = q2_bool.astype(int)
q3_bool = q3_bool.astype(int)
q4_bool = q4_bool.astype(int)



LoadError: UndefVarError: structure_class not defined

In [ ]:
q1_data = den.copy()
q1_data[np.where (structure_class!=0)] = 0.0

q2_data = den.copy()
q2_data[np.where (structure_class!=1)] = 0.0

q3_data = den.copy()
q3_data[np.where (structure_class!=2)] = 0.0

q4_data = den.copy()
q4_data[np.where (structure_class!=3)] = 0.0

In [2]:
include("MyModule.jl")

LoadError: ArgumentError: Package run_tagging not found in current path:
- Run `import Pkg; Pkg.add("run_tagging")` to install the run_tagging package.
